Test basic text task with reformer enc_dec:  
https://github.com/lucidrains/reformer-pytorch#reformer-encoder-decoder-architecture

In [16]:
from fastai.text.all import *
from reformer_pytorch import ReformerEncDec
torch.cuda.set_device(3)

# Use IMDB_sample


In [17]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')
df.head(2)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False


In [18]:
#mostly long texts:
[(sl, (df['text'].apply(len)>sl).mean()) for sl in range(128,1024+1, 128)]

[(128, 1.0),
 (256, 0.983),
 (384, 0.946),
 (512, 0.907),
 (640, 0.852),
 (768, 0.693),
 (896, 0.589),
 (1024, 0.511)]

In [19]:
bs = 8
seq_len = 128

> is eos correctly applied?

In [20]:
def add_eos(text):
    return text + f' {EOS}'

In [21]:
dls = DataBlock(blocks=TextBlock.from_df('text', is_lm=True, rules=[add_eos], seq_len=seq_len),
                get_x=ColReader('text'), 
                splitter=RandomSplitter(valid_pct=0.1)
               ).dataloaders(df, bs=bs)

In [22]:
dls.show_batch(max_n=2)

,text,text_
0,""" How To Lose Friends & Alienate People "" is not based on xxunk Woods ' xxunk . It is a mediocre romantic comedy based on Toby Young 's book on his experiences working as a journalist covering xxunk . The film stars Simon Pegg as Sidney Young , a xxunk British journalist who takes a job in an illustrious celebrity magazine in New York . Young is xxunk in getting caught up all type of shenanigans to xxunk all around him , hence movie title . He is xxunk , daring , and moronic . But nevertheless for some very bizarre reason , he is a somewhat likable character . Sidney xxunk a fellow journalist , the composed Alison xxunk , played quite xxunk by xxunk xxunk","How To Lose Friends & Alienate People "" is not based on xxunk Woods ' xxunk . It is a mediocre romantic comedy based on Toby Young 's book on his experiences working as a journalist covering xxunk . The film stars Simon Pegg as Sidney Young , a xxunk British journalist who takes a job in an illustrious celebrity magazine in New York . Young is xxunk in getting caught up all type of shenanigans to xxunk all around him , hence movie title . He is xxunk , daring , and moronic . But nevertheless for some very bizarre reason , he is a somewhat likable character . Sidney xxunk a fellow journalist , the composed Alison xxunk , played quite xxunk by xxunk xxunk ."
1,"final moments and , of course , steals the show . Dancy and Wilson are well worth the price of xxunk too.<br /><br />As with "" Away from Her , "" "" Evening "" stays with you at length , xxunk a re - thinking its story and characters , and re - experiencing the emotions it raises . At two hours , the film runs a bit long , but the way it stays with you xxunk is welcome among the many movies that go cold long before your popcorn . xxeos I had heard good things about this film and was , you guessed it , a bit disappointed . Reese xxunk is as promised surprisingly good , surprisingly confident , at a young age ;","moments and , of course , steals the show . Dancy and Wilson are well worth the price of xxunk too.<br /><br />As with "" Away from Her , "" "" Evening "" stays with you at length , xxunk a re - thinking its story and characters , and re - experiencing the emotions it raises . At two hours , the film runs a bit long , but the way it stays with you xxunk is welcome among the many movies that go cold long before your popcorn . xxeos I had heard good things about this film and was , you guessed it , a bit disappointed . Reese xxunk is as promised surprisingly good , surprisingly confident , at a young age ; really"


The vocab. 1 is pad, 3 is eos.

In [23]:
L(dls.vocab), PAD, EOS

((#7704) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the'...],
 'xxpad',
 'xxeos')

## test for eos

In [24]:
# for xb, _ in dls.train:
#     if (xb==3).sum() > 0:
#         for b in xb:
#             if (b==3).sum()>0:
#                 print(b)

# Model

From the paper - synthetic task specification. Try similar for text task:
>To make it easy and fast to train but similar to models used in NLP, we use a 1-layer Transformer with dmodel = dff = 256 and 4 heads. We train it for 150K steps in 4 different settings: with full attention, LSH attention with nrounds = 1, nrounds = 2 and nrounds = 4.

## model spec

In [25]:
# model = ReformerLM(
#     num_tokens= 20000,
#     dim = 1024,
#     depth = 12,
#     max_seq_len = 8192,
#     heads = 8,
#     lsh_dropout = 0.1,
#     ff_dropout = 0.1,
#     post_attn_dropout = 0.1,
#     layer_dropout = 0.1,  # layer dropout from 'Reducing Transformer Depth on Demand' paper
#     causal = True,        # auto-regressive or not
#     bucket_size = 64,     # average size of qk per bucket, 64 was recommended in paper
#     n_hashes = 4,         # 4 is permissible per author, 8 is the best but slower
#     emb_dim = 128,        # embedding factorization for further memory savings
#     dim_head = 64,        # be able to fix the dimension of each head, making it independent of the embedding dimension and the number of heads
#     ff_chunks = 200,      # number of chunks for feedforward layer, make higher if there are memory issues
#     attn_chunks = 8,      # process lsh attention in chunks, only way for memory to fit when scaling to 16k tokens
#     num_mem_kv = 128,       # persistent learned memory key values, from all-attention paper
#     twin_attention = False, # both branches of the reversible network will be attention
#     full_attn_thres = 1024, # use full attention if context length is less than set value
#     reverse_thres = 1024,   # turn off reversibility for 2x speed for sequence lengths shorter or equal to the designated value
#     use_scale_norm = False,  # use scale norm from 'Transformers without tears' paper
#     use_rezero = False,      # remove normalization and use rezero from 'ReZero is All You Need'
#     one_value_head = False,  # use one set of values for all heads from 'One Write-Head Is All You Need'
#     weight_tie = False,           # tie parameters of each layer for no memory per additional depth
#     weight_tie_embedding = False, # use token embedding for projection of output, some papers report better results
#     n_local_attn_heads = 2,       # many papers suggest mixing local attention heads aids specialization and improves on certain tasks
#     pkm_layers = (4,7),           # specify layers to use product key memory. paper shows 1 or 2 modules near the middle of the transformer is best
#     pkm_num_keys = 128,           # defaults to 128, but can be increased to 256 or 512 as memory allows
#     use_full_attn = True    # only turn on this flag to override and turn on full attention for all sequence lengths. for comparison with LSH to show that it is working
# ).cuda()

# Training

Do the training loop from scratch:

In [26]:
num_tokens = len(dls.vocab) # number of tokens in dataloader
n_layers = 1
n_heads = 4

>how to set ff-dim?

In [27]:
# Note: enc_dec parameters must be specified with 'enc' or 'dec' suffix

model = ReformerEncDec(
    dim = 256,
    
    enc_heads = 4, ## ok??
    enc_num_tokens = num_tokens,
    enc_depth = n_layers,
    enc_max_seq_len = seq_len,
    
    dec_num_tokens = num_tokens,
    dec_depth = n_layers,
    dec_heads = n_heads, ## ok??
    dec_max_seq_len = seq_len
).cuda()

In [28]:
#Num params:
sum([d.flatten().shape[0] for d in model.parameters()])

7422232

Use a basic optimiser

In [29]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

**Note: loss function is built in to model**  
Training loop is a bit atypical.

In [30]:
for epoch in range(3): 
    
    running_loss = []
    
    for i, (xb, yb) in enumerate(dls.train):
        
        #the model crashes on sequences other than 128
        if xb.shape[1] != 128:
            continue
            
        # need to pass a mask, check for token id 1 - the pad token in fastai
        input_mask = xb==1
                
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # this is how the loss is calculated in the documentation - forward and loss in combined in model's forward pass
        loss = model(xb, yb, return_loss = True, enc_input_mask = input_mask) 
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss.append(loss.item())
        if i%50 == 0:
            print(f'epoch: {epoch}, iteration: {i}, losss: {np.mean(running_loss)}')

epoch: 0, iteration: 0, losss: 9.089465141296387
epoch: 0, iteration: 50, losss: 8.882287455540077
epoch: 0, iteration: 100, losss: 8.603285345700709
epoch: 0, iteration: 150, losss: 8.36748148116055
epoch: 0, iteration: 200, losss: 8.186546071844907
epoch: 1, iteration: 0, losss: 7.306553840637207
epoch: 1, iteration: 50, losss: 7.287906702827005
epoch: 1, iteration: 100, losss: 7.218148396746947
epoch: 1, iteration: 150, losss: 7.168114295858421
epoch: 1, iteration: 200, losss: 7.122259317938961
epoch: 2, iteration: 0, losss: 6.853013515472412
epoch: 2, iteration: 50, losss: 6.8687695241441915
epoch: 2, iteration: 100, losss: 6.852435890990908
epoch: 2, iteration: 150, losss: 6.8119382637226025
epoch: 2, iteration: 200, losss: 6.794969864745639


# Evaluate

# Interpret